In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import math
import statsmodels.api as sm
import patsy as pt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
!gdown --folder 18tPh3vljXUWLlfQV4GDdLlrvKJpuSRcU

Retrieving folder list
Processing file 1-N77AoUP-odeDszvUp3A5hCtnkDGiG33 plus1.csv
Processing file 1-Eda1OsxgGHMirvIT7t3U0cCu8pGOBC- plus2.csv
Processing file 1-8TZYiQdfYMBVqcrZc1WSBhKUZCeasV_ PRICES_HISTORY_DAILY.csv
Processing file 1-LwViKXDGAgBJ4v1VJhEOO9FnGmb0kZK start_without_promo.csv
Processing file 1-BECSHBIgAJWyOAVKTtBbtdZorQ6CamM TRANSACTIONS_HISTORY.csv
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1-N77AoUP-odeDszvUp3A5hCtnkDGiG33
To: /content/data/plus1.csv
100% 7.13k/7.13k [00:00<00:00, 12.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-Eda1OsxgGHMirvIT7t3U0cCu8pGOBC-
To: /content/data/plus2.csv
100% 7.05k/7.05k [00:00<00:00, 12.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-8TZYiQdfYMBVqcrZc1WSBhKUZCeasV_
To: /content/data/PRICES_HISTORY_DAILY.csv
100% 8.44M/8.44M [00:00<00:00, 51.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=

In [3]:
data1=pd.read_csv('data/PRICES_HISTORY_DAILY.csv', sep = ';')
data2=pd.read_csv('data/TRANSACTIONS_HISTORY.csv')
data2 = data2.drop('TRANSACTION_ID', 1)
data2 = data2.drop('CUSTOMER_ID', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [4]:
tmp = pd.merge(data1, data2, how='inner', on=['PRODUCT_ID', 'DATE', 'STORE_ID']).drop_duplicates(subset=['PRODUCT_ID', 'DATE', 'STORE_ID'], keep="first")

In [5]:
tmp0=tmp
tmp0=tmp0.query('PRICE_TYPE != 0')
tmp0.to_csv('start_without_promo.csv', index=False)

In [6]:
tmp = tmp.replace({'STORE_ID':{'DC01':0, 'DC02':1}})
tmp = tmp.replace({'PRICE_TYPE':{'PROMO':0, 'REGULAR':1}})
tmp = tmp.drop(['DATE', 'PRICE_TYPE'], 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
df=tmp
df.loc[df['STORE_ID']==0, 'PRODUCT_ID'] *= 10
df.loc[df['STORE_ID']==1, 'PRODUCT_ID'] *= 10
df.loc[df['STORE_ID']==1, 'PRODUCT_ID'] += 1
df_plus=df
df_plus = df_plus.assign(price_plus=df.PRICE+5)
df_plus=df_plus.drop('PRICE', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  import sys


In [8]:
coef_dict = dict()
prods_list = []
coef_list = []
for prod in tmp['PRODUCT_ID'].unique():
    df=tmp[tmp['PRODUCT_ID']==prod]
    X = df['PRICE'].values.reshape(-1,1)
    y = df['QNT']

    reg_qnt0 = LinearRegression()
    reg_qnt0.fit(X, y)
    coef_dict[prod] = [reg_qnt0.intercept_, reg_qnt0.coef_[0]]
    coef_dict

In [9]:
ans = pd.DataFrame(list(coef_dict.items()),
                   columns=['pr', 'coef'])
ans = ans.assign(PRODUCT_ID = ans.pr // 10 )
ans = ans.assign(STORE_ID = ans.pr % 10)
ans=ans.drop(['pr'], 1)
ans.to_csv('first.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """


In [10]:
df_log = tmp0
df_log = df_log.assign(LOG_PRICE=np.log(df_log.PRICE))

In [11]:
coef_dict_log = dict()
prods_list = []
coef_list = []
for prod in df_log['PRODUCT_ID'].unique():
    df=df_log[tmp0['PRODUCT_ID']==prod]
    X = df['LOG_PRICE'].values.reshape(-1,1)
    y = df['QNT']

    reg_qnt0 = LinearRegression()
    reg_qnt0.fit(X, y)
    coef_dict_log[prod] = [reg_qnt0.intercept_, reg_qnt0.coef_[0]]
answer_log = pd.DataFrame(list(coef_dict_log.items()),
                   columns=['pr', 'coef'])
answer_log = answer_log.assign(PRODUCT_ID = answer_log.pr // 10 )
answer_log = answer_log.assign(STORE_ID = answer_log.pr % 10)
answer_log=answer_log.drop(['pr'], 1)
answer_log.to_csv('fourth.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  app.launch_new_instance()


In [12]:
coef_dict = dict()
prods_list = []
coef_list = []
for prod in tmp0['PRODUCT_ID'].unique():
    df=tmp0[tmp0['PRODUCT_ID']==prod]
    X = df['PRICE'].values.reshape(-1,1)
    y = df['QNT']

    reg_qnt0 = LinearRegression()
    reg_qnt0.fit(X, y)
    coef_dict[prod] = [reg_qnt0.intercept_, reg_qnt0.coef_[0]]
answer = pd.DataFrame(list(coef_dict.items()),
                   columns=['pr', 'coef'])
answer = answer.assign(PRODUCT_ID = answer.pr // 10 )
answer = answer.assign(STORE_ID = answer.pr % 10)
answer=answer.drop(['pr'], 1)
answer.to_csv('without_promo_first.csv', index=False) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  app.launch_new_instance()


In [13]:
coef_dict_log = dict()
prods_list = []
coef_list = []
for prod in df_log['PRODUCT_ID'].unique():
    df=df_log[tmp0['PRODUCT_ID']==prod]
    X = df['LOG_PRICE'].values.reshape(-1,1)
    y = df['QNT']

    reg_qnt0 = LinearRegression()
    reg_qnt0.fit(X, y)
    coef_dict_log[prod] = [reg_qnt0.intercept_, reg_qnt0.coef_[0]]
answer_log = pd.DataFrame(list(coef_dict_log.items()),
                   columns=['pr', 'coef'])
answer_log = answer_log.assign(PRODUCT_ID = answer_log.pr // 10 )
answer_log = answer_log.assign(STORE_ID = answer_log.pr % 10)
answer_log=answer_log.drop(['pr'], 1)
answer_log.to_csv('without_promo_log_first')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  app.launch_new_instance()


In [14]:
coef_dict = dict()
prods_list = []
coef_list = []
for prod in df_plus['PRODUCT_ID'].unique():
    df1=df_plus[df_plus['PRODUCT_ID']==prod]
    X = df1['price_plus'].values.reshape(-1,1)
    y = df1['QNT']

    reg_qnt0 = LinearRegression()
    reg_qnt0.fit(X, y)
    coef_dict[prod] = [reg_qnt0.intercept_, reg_qnt0.coef_[0]]
    coef_dict
ans = pd.DataFrame(list(coef_dict.items()),
                   columns=['pr', 'coef'])
ans = ans.assign(PRODUCT_ID = ans.pr // 10 )
ans = ans.assign(STORE_ID = ans.pr % 10)
ans=ans.drop(['pr'], 1)
c = np.vstack(ans['coef'])
ans['coef_b'] = c[:, 0]
ans['coef_k'] = c[:, 1]
ans = ans.drop('coef', axis=1)
ans.to_csv('plus1.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [15]:
df_plus_log = df_plus
df_plus_log = df_plus_log.assign(LOG_PRICE=np.log(df_plus_log.price_plus))

In [16]:
coef_dict_log = dict()
prods_list = []
coef_list = []
for prod in df_plus_log['PRODUCT_ID'].unique():
    df=df_plus_log[df_plus_log['PRODUCT_ID']==prod]
    X = df['LOG_PRICE'].values.reshape(-1,1)
    y = df['QNT']
    reg_qnt0 = LinearRegression()
    reg_qnt0.fit(X, y)
    coef_dict_log[prod] = [reg_qnt0.intercept_, reg_qnt0.coef_[0]]
answer_log = pd.DataFrame(list(coef_dict_log.items()),
                   columns=['pr', 'coef'])
answer_log = answer_log.assign(PRODUCT_ID = answer_log.pr // 10 )
answer_log = answer_log.assign(STORE_ID = answer_log.pr % 10)
answer_log=answer_log.drop(['pr'], 1)
c = np.vstack(answer_log['coef'])
answer_log['coef_b'] = c[:, 0]
answer_log['coef_k'] = c[:, 1]
answer_log = answer_log.drop('coef', axis=1)
answer_log.to_csv('plus2.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  from ipykernel import kernelapp as app


In [17]:
data4 = pd.read_csv('plus2.csv', sep = ',')
data5 = pd.read_csv('plus1.csv', sep = ',')

In [18]:
data5['coef_k'].loc[data5['coef_k'] > 2.8] = -1.633449
data5.to_csv('1605(3).csv', index=False)
data5['coef_k'].loc[data5['coef_k'] > 0.000000] = -1.633449
data5.to_csv('1605(2).csv', index=False)
data5['coef_k'].loc[data5['coef_k'] >= 0.000000] = -1.633449
data5.to_csv('1605(1).csv', index=False)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [19]:
tmp1=pd.read_csv('data/start_without_promo.csv', sep = ',')

In [20]:
coef_dict = dict()
prods_list = []
coef_list = []
for prod in tmp['PRODUCT_ID'].unique():
    df=tmp[tmp['PRODUCT_ID']==prod]
    X = df['PRICE'].values.reshape(-1,1)
    y = df['QNT']
    reg_qnt0 = Ridge(alpha = 0.2)
    reg_qnt0.fit(X, y)
    coef_dict[prod] = [reg_qnt0.intercept_, reg_qnt0.coef_[0]]
ans = pd.DataFrame(list(coef_dict.items()),
                   columns=['pr', 'coef'])
ans = ans.assign(PRODUCT_ID = ans.pr // 10 )
ans = ans.assign(STORE_ID = ans.pr % 10)
ans=ans.drop(['pr'], 1)
c = np.vstack(ans['coef'])
ans['coef_b'] = c[:, 0]
ans['coef_k'] = c[:, 1]
ans = ans.drop('coef', axis=1)
ans.to_csv('Ridge_first.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  from ipykernel import kernelapp as app


In [21]:
coef_dict = dict()
prods_list = []
coef_list = []
for prod in tmp['PRODUCT_ID'].unique():
    df=tmp[tmp['PRODUCT_ID']==prod]
    X = df['PRICE'].values.reshape(-1,1)
    y = df['QNT']
    reg_qnt0 = ElasticNet(normalize=True,alpha=0.001,l1_ratio=0.75)
    reg_qnt0.fit(X, y)
    coef_dict[prod] = [reg_qnt0.intercept_, reg_qnt0.coef_[0]]
ans = pd.DataFrame(list(coef_dict.items()),
                   columns=['pr', 'coef'])
ans = ans.assign(PRODUCT_ID = ans.pr // 10 )
ans = ans.assign(STORE_ID = ans.pr % 10)
ans=ans.drop(['pr'], 1)
c = np.vstack(ans['coef'])
ans['coef_b'] = c[:, 0]
ans['coef_k'] = c[:, 1]
ans = ans.drop('coef', axis=1)
ans.to_csv('ElasticNet_first.csv', index=False)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), ElasticNet())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to original_alpha * np.sqrt(n_samples) if l1_ratio is 1, and to original_alpha * n_samples if l1_ratio is 0. For other values of l1_ratio, no analytic formula is available.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If

In [22]:
coef_dict = dict()
prods_list = []
coef_list = []
for prod in tmp['PRODUCT_ID'].unique():
    df=tmp[tmp['PRODUCT_ID']==prod]
    X = df['PRICE'].values.reshape(-1,1)
    y = df['QNT']
    reg_qnt0 = Lasso(alpha = 0.0001)
    reg_qnt0.fit(X, y)
    coef_dict[prod] = [reg_qnt0.intercept_, reg_qnt0.coef_[0]]
ans = pd.DataFrame(list(coef_dict.items()),
                   columns=['pr', 'coef'])
ans = ans.assign(PRODUCT_ID = ans.pr // 10 )
ans = ans.assign(STORE_ID = ans.pr % 10)
ans=ans.drop(['pr'], 1)
c = np.vstack(ans['coef'])
ans['coef_b'] = c[:, 0]
ans['coef_k'] = c[:, 1]
ans = ans.drop('coef', axis=1)
ans.to_csv('Lasso_first.csv', index=False)
ans

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisa

,PRODUCT_ID,STORE_ID,coef_b,coef_k
0,3046357,0,0.751724,0.128407
1,3046358,0,1.482198,-0.061359
2,3047286,0,0.896222,0.092763
3,3047287,0,1.121789,0.079844
4,3047290,0,1.446377,-0.042932
...,...,...,...,...
142,3396472,1,1.000000,0.000000
143,3396473,1,8.476413,-1.878127
144,3396474,1,38.565535,-8.889718
145,3396475,1,1.000000,0.000000
